In [1]:
import sys
sys.path.append('../')
from helper import Data
import numpy as np

In [2]:
data = Data()

d = Data()

d.fetch_data()
df = d.preprocess_data(d.df_us_cases)
df = d.daily_data(df)

In [23]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
num_estimators = 20
scaler = MinMaxScaler()
train = df['Total'].tolist()
train = np.array(train).reshape((-1, 1))
train = scaler.fit_transform(train)

In [24]:
train = train.ravel()

In [115]:
lookback = 7
forecast_interval = 7
x_train, y_train, pred_train = [], [], []
for i in range(len(train) - lookback - forecast_interval + 1):
    x = train[i:i + lookback]
    x_train.append(x)
    y_train.append(train[i + lookback:i + lookback + forecast_interval])
    
for i in range(0,len(train)-lookback+1):
    pred_train.append(train[i:i+lookback])
    
pred_train = np.array(pred_train)
pred_train = pred_train.reshape((pred_train.shape[0], pred_train.shape[1],1))

In [116]:
print(np.array(x_train).shape)
print(np.array(y_train).shape)
print(len(df))
print(np.array(pred_train).shape)

(163, 7)
(163, 7)
176
(170, 7, 1)


In [117]:
print(x_train[0])
print(train[:20])
print(y_train[0])

[0.0000000e+00 1.2944146e-05 0.0000000e+00 3.8832438e-05 0.0000000e+00
 0.0000000e+00 0.0000000e+00]
[0.0000000e+00 1.2944146e-05 0.0000000e+00 3.8832438e-05 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 2.5888292e-05 1.2944146e-05
 0.0000000e+00 3.8832438e-05 0.0000000e+00 0.0000000e+00 0.0000000e+00
 0.0000000e+00 0.0000000e+00 0.0000000e+00 0.0000000e+00 1.2944146e-05]
[0.0000000e+00 2.5888292e-05 1.2944146e-05 0.0000000e+00 3.8832438e-05
 0.0000000e+00 0.0000000e+00]


In [31]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], 1))

In [94]:
x_train.shape

(161, 7, 1)

In [35]:
x_test = np.array(y_train[-1]).reshape((1,x_train.shape[1],1))
np.concatenate([x_train, x_test]).shape

(162, 7, 1)

In [99]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

model = Sequential()
model.add(LSTM(50, input_shape=(7,1)))
# model.add(LSTM(50, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(7))

model.compile(loss='mean_squared_error', optimizer='adam')
y_hat = []
pred_hat = []
for _ in range(20):
    model.fit(x_train, y_train, epochs=100, verbose=0, shuffle=True)
    x_test = np.array(y_train[-1]).reshape((1,x_train.shape[1],1))
    y_hat.append(scaler.inverse_transform(model.predict(pred_train)))
    pred_hat.append(scaler.inverse_transform(model.predict(x_test)))

In [140]:
x_test = np.array(y_train[-1]).reshape((1,x_train.shape[1],1))
scaler.inverse_transform(model.predict(x_test))

array([[65695.48 , 64244.9  , 68901.695, 70802.55 , 71321.46 , 77086.695,
        75212.48 ]], dtype=float32)

In [97]:
y_hat = np.array(y_hat)
pred_hat = np.array(pred_hat)

In [98]:
y_hat.shape

(20, 1, 7)

In [75]:
ans = []
for i in range(20):
    ans.append(y_hat[i,0:y_hat.shape[1]:7].ravel())
    

In [76]:
ans = np.array(ans)

In [81]:
ans.shape

(20, 168)

In [86]:
low, high = [], []
pred = []
for i in range(ans.shape[1]):
    low.append(max(ans[:,i].min(),0))
    high.append(ans[:,i].max())
    pred.append(ans[:,i].mean())

In [87]:
low

[0,
 0,
 0,
 0,
 4.687067,
 188.42146,
 270.80804,
 0,
 0,
 0,
 0,
 5.898118,
 189.70044,
 272.2925,
 0,
 0,
 0,
 0,
 2.771488,
 187.3785,
 269.57397,
 0,
 0,
 0,
 0,
 1.5500762,
 186.10298,
 268.61792,
 0,
 0,
 0,
 0,
 2.0940132,
 186.38617,
 269.08356,
 0,
 0,
 0,
 0,
 77.79967,
 267.90158,
 358.0043,
 217.29588,
 299.1132,
 490.7134,
 446.47952,
 699.8378,
 943.95013,
 1191.0842,
 1767.3872,
 2213.6453,
 2523.612,
 2934.102,
 3148.1187,
 3738.987,
 4030.8098,
 9382.222,
 9548.464,
 9745.882,
 10345.816,
 10102.783,
 11754.646,
 10519.738,
 22627.232,
 23100.938,
 23201.543,
 24131.918,
 23259.805,
 25439.652,
 23709.012,
 31114.68,
 32155.928,
 31583.46,
 29241.5,
 28969.568,
 29382.729,
 31258.291,
 28649.248,
 28971.496,
 27235.0,
 23437.523,
 22690.871,
 22914.29,
 25246.158,
 27773.21,
 27189.164,
 25864.016,
 23015.244,
 22715.857,
 23305.777,
 25397.605,
 27365.34,
 28061.412,
 26710.271,
 21822.986,
 19321.143,
 18913.928,
 21694.031,
 25755.137,
 26499.111,
 25379.852,
 2143

In [114]:
x_test = np.array(x_train[-1]).reshape((1,x_train.shape[1],1))
for i in range(100):
    y_pred = model.predict(x_test)
    x_test = x_test.ravel()[1:]
    x_test = np.append(x_test,y_pred.ravel())
    x_test = x_test.reshape((1,x_train.shape[1],1))
    print(scaler.inverse_transform(y_pred))

[[59820.402]]
[[59824.137]]
[[61667.137]]
[[63817.77]]
[[64730.633]]
[[64499.453]]
[[63712.95]]
[[63117.688]]
[[63088.355]]
[[63485.418]]
[[63933.062]]
[[64139.03]]
[[64061.11]]
[[63847.504]]
[[63686.66]]
[[63671.883]]
[[63767.984]]
[[63878.88]]
[[63929.777]]
[[63907.152]]
[[63849.773]]
[[63806.734]]
[[63802.21]]
[[63827.46]]
[[63856.816]]
[[63869.832]]
[[63863.17]]
[[63847.406]]
[[63835.76]]
[[63834.62]]
[[63841.48]]
[[63849.348]]
[[63852.758]]
[[63850.875]]
[[63846.582]]
[[63843.457]]
[[63843.188]]
[[63845.05]]
[[63847.168]]
[[63848.07]]
[[63847.543]]
[[63846.383]]
[[63845.54]]
[[63845.473]]
[[63845.99]]
[[63846.56]]
[[63846.8]]
[[63846.65]]
[[63846.336]]
[[63846.11]]
[[63846.094]]
[[63846.234]]
[[63846.39]]
[[63846.453]]
[[63846.418]]
[[63846.34]]
[[63846.277]]
[[63846.26]]
[[63846.297]]
[[63846.336]]
[[63846.348]]
[[63846.34]]
[[63846.32]]
[[63846.312]]
[[63846.312]]
[[63846.32]]
[[63846.33]]
[[63846.33]]
[[63846.32]]
[[63846.316]]
[[63846.316]]
[[63846.316]]
[[63846.312]]
[[63846.

In [111]:
x_test.ravel()+[1]
np.append(x_test.ravel(),[1])

array([2.43350841, 2.34859026, 2.62642808, 2.82855719, 2.51310422,
       1.        ])

In [143]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,32672,32951,33190,33384,33594,33908,34194,34366,34451,34455
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,2819,2893,2964,3038,3106,3188,3278,3371,3454,3571
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,15500,15941,16404,16879,17348,17808,18242,18712,19195,19689
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,858
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,346,346,346,386,386,396,458,462,506,525
